In [164]:
%matplotlib inline   

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from scipy import stats, integrate
import seaborn as sns
from sklearn.pipeline import Pipeline

sns.set(style="whitegrid")


In [316]:
ruta_train='C:/Users/Juan Diego Bernate V/Downloads/Python/Practica/Titanic/train.csv'
ds=pd.read_csv(ruta_train)


In [131]:
#passengerId = test.PassengerId
#train.head()


In [179]:
#ds= train.append(test, ignore_index=True)

#train_idx = len(train)
#test_idx = len(train) - len(test)
ds.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [317]:
# Identificando los titulos de los pasajeros
ds['titulo'] = ds.Name.apply(lambda name: name.split(',')[1].split('.')[0].strip())

In [318]:
ds.titulo.unique()
#Corregimos los titulos 
# normalize the titles
titulos_arreglados = {
    "Capt":       "Oficial",
    "Col":        "Oficial",
    "Major":      "Oficial",
    "Jonkheer":   "Realeza",
    "Don":        "Realeza",
    "Sir" :       "Realeza",
    "Dr":         "Oficial",
    "Rev":        "Oficial",
    "the Countess":"Realeza",
    "Dona":       "Realeza",
    "Mme":        "Senora",
    "Mlle":       "Senorita",
    "Ms":         "Senora",
    "Mr" :        "Senor",
    "Mrs" :       "Senora",
    "Miss" :      "Senorita",
    "Master" :    "Master",
    "Lady" :      "Realeza"
}

#Reemplazamos en el dataset unido
ds.titulo = ds.titulo.map(titulos_arreglados)

In [319]:
ds.isnull().sum()

group = ds.groupby(['Sex','Pclass', 'titulo'])  

# La mediana por grupo
group.Age.median()

Sex     Pclass  titulo  
female  1       Oficial     49.0
                Realeza     40.5
                Senora      40.0
                Senorita    30.0
        2       Senora      31.5
                Senorita    24.0
        3       Senora      31.0
                Senorita    18.0
male    1       Master       4.0
                Oficial     51.0
                Realeza     40.0
                Senor       40.0
        2       Master       1.0
                Oficial     46.5
                Senor       31.0
        3       Master       4.0
                Senor       26.0
Name: Age, dtype: float64

In [320]:
## Rellenamos los valores nulos con la media en la edad  y en los demás con la moda (VERSION INICAL JD)
#ds1=ds.fillna(ds.median())
#ds1['Cabin'].fillna(ds1['Cabin'].mode()[0], inplace=True)
#ds1['Embarked'].fillna(ds1['Embarked'].mode()[0], inplace=True)

ds.Age = group.Age.apply(lambda x: x.fillna(x.median()))
ds['Fare']=ds['Fare'].fillna(ds['Fare'].median())


In [321]:
ds['Embarked'].fillna(ds['Embarked'].mode()[0], inplace=True)

In [322]:
ds.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
titulo           0
dtype: int64

In [323]:
ds.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,titulo
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Senor
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Senora
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Senorita
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Senora
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Senor


In [324]:
d = {1:'1st',2:'2nd',3:'3rd'} 
ds['Pclass'] = ds['Pclass'].map(d)


In [325]:
Categoricas = ds[['Pclass','Sex','Embarked','titulo']]
dummies = pd.get_dummies(Categoricas,drop_first=True)
ds=ds.drop(['Pclass','Sex','Embarked','titulo'],1)

In [326]:
ds['familia']=ds['SibSp']+ds['Parch']+1

In [327]:
ds=pd.concat([ds,dummies],axis=1)

In [218]:
ds.head()

,PassengerId,Survived,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,familia,Pclass_2nd,Pclass_3rd,Sex_male,Embarked_Q,Embarked_S,titulo_Oficial,titulo_Realeza,titulo_Senor,titulo_Senora,titulo_Senorita
0,1,0,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,2,0,1,1,0,1,0,0,1,0,0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,2,0,0,0,0,0,0,0,0,1,0
2,3,1,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,1,0,1,0,0,1,0,0,0,0,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,2,0,0,0,0,1,0,0,0,1,0
4,5,0,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,1,0,1,1,0,1,0,0,1,0,0


In [328]:
ds=ds.drop(['Pclass','Sex','Embarked'],1)

KeyError: "labels ['Pclass' 'Sex' 'Embarked'] not contained in axis"

In [329]:
ds.head()

,PassengerId,Survived,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,familia,Pclass_2nd,Pclass_3rd,Sex_male,Embarked_Q,Embarked_S,titulo_Oficial,titulo_Realeza,titulo_Senor,titulo_Senora,titulo_Senorita
0,1,0,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,2,0,1,1,0,1,0,0,1,0,0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,2,0,0,0,0,0,0,0,0,1,0
2,3,1,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,1,0,1,0,0,1,0,0,0,0,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,2,0,0,0,0,1,0,0,0,1,0
4,5,0,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,1,0,1,1,0,1,0,0,1,0,0


In [87]:
#VOLVEMOS A SEPARAR
#Train = ds[ :train_idx]
#Test = ds[test_idx: ]

In [97]:
#ds1=Train
#Ent=Test

In [330]:
ds.head()

,PassengerId,Survived,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,familia,Pclass_2nd,Pclass_3rd,Sex_male,Embarked_Q,Embarked_S,titulo_Oficial,titulo_Realeza,titulo_Senor,titulo_Senora,titulo_Senorita
0,1,0,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,2,0,1,1,0,1,0,0,1,0,0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,2,0,0,0,0,0,0,0,0,1,0
2,3,1,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,1,0,1,0,0,1,0,0,0,0,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,2,0,0,0,0,1,0,0,0,1,0
4,5,0,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,1,0,1,1,0,1,0,0,1,0,0


In [116]:
#ds.Survived = ds.Survived.astype(int)


In [331]:
ds=ds.drop(['PassengerId','Name','Cabin','Ticket'],1)

In [332]:
ds.head()

,Survived,Age,SibSp,Parch,Fare,familia,Pclass_2nd,Pclass_3rd,Sex_male,Embarked_Q,Embarked_S,titulo_Oficial,titulo_Realeza,titulo_Senor,titulo_Senora,titulo_Senorita
0,0,22.0,1,0,7.2500,2,0,1,1,0,1,0,0,1,0,0
1,1,38.0,1,0,71.2833,2,0,0,0,0,0,0,0,0,1,0
2,1,26.0,0,0,7.9250,1,0,1,0,0,1,0,0,0,0,1
3,1,35.0,1,0,53.1000,2,0,0,0,0,1,0,0,0,1,0
4,0,35.0,0,0,8.0500,1,0,1,1,0,1,0,0,1,0,0


In [333]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier


In [334]:
np.random.seed(123)
from sklearn.model_selection import train_test_split

X = ds.drop(['Survived'], axis=1)
y = ds['Survived']


In [335]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

print(X_train.shape[0], X_test.shape[0])


712 179


In [336]:
parameter_gridRF = {
   'n_estimators' : [1000, 2000, 2500, 3000],
   'max_features': [0.5, 0.8, None, 'sqrt'],
    'max_depth': [2, 3, 5, 10, 15, 20, None]
       }

parameter_gridGBR = {
    'learning_rate' : [0.1, 0.05, 0.01],
       'n_estimators' : [ 1000, 2000],
    'loss': ['deviance', 'exponential'],
    'max_depth': [2, 3, 5, 10, 15, 20, None]
    }

grid_search1 = GridSearchCV(RandomForestClassifier(n_jobs=-1), parameter_gridRF,
                            cv=5, verbose=3, scoring ='roc_auc')

grid_search2 = GridSearchCV(GradientBoostingClassifier(), parameter_gridGBR,
                            cv=5, verbose=3, scoring ='roc_auc')

In [337]:
grid_search1.fit(X_test, y_test)

Fitting 5 folds for each of 112 candidates, totalling 560 fits
[CV] n_estimators=1000, max_features=0.5, max_depth=2 ................
[CV]  n_estimators=1000, max_features=0.5, max_depth=2, score=0.8095238095238095, total=   1.4s
[CV] n_estimators=1000, max_features=0.5, max_depth=2 ................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.6s remaining:    0.0s


[CV]  n_estimators=1000, max_features=0.5, max_depth=2, score=0.9587301587301589, total=   1.5s
[CV] n_estimators=1000, max_features=0.5, max_depth=2 ................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.5s remaining:    0.0s


[CV]  n_estimators=1000, max_features=0.5, max_depth=2, score=0.8158730158730159, total=   1.7s
[CV] n_estimators=1000, max_features=0.5, max_depth=2 ................
[CV]  n_estimators=1000, max_features=0.5, max_depth=2, score=0.888888888888889, total=   1.4s
[CV] n_estimators=1000, max_features=0.5, max_depth=2 ................
[CV]  n_estimators=1000, max_features=0.5, max_depth=2, score=0.9081632653061225, total=   1.6s
[CV] n_estimators=2000, max_features=0.5, max_depth=2 ................
[CV]  n_estimators=2000, max_features=0.5, max_depth=2, score=0.7999999999999999, total=   3.1s
[CV] n_estimators=2000, max_features=0.5, max_depth=2 ................
[CV]  n_estimators=2000, max_features=0.5, max_depth=2, score=0.9682539682539683, total=   2.7s
[CV] n_estimators=2000, max_features=0.5, max_depth=2 ................
[CV]  n_estimators=2000, max_features=0.5, max_depth=2, score=0.8190476190476191, total=   3.0s
[CV] n_estimators=2000, max_features=0.5, max_depth=2 ................

...

[CV]  n_estimators=2000, max_features=None, max_depth=20, score=0.8507936507936508, total=   3.2s
[CV] n_estimators=2000, max_features=None, max_depth=20 ..............
[CV]  n_estimators=2000, max_features=None, max_depth=20, score=0.9873015873015873, total=   2.7s
[CV] n_estimators=2000, max_features=None, max_depth=20 ..............
[CV]  n_estimators=2000, max_features=None, max_depth=20, score=0.8428571428571429, total=   2.9s
[CV] n_estimators=2000, max_features=None, max_depth=20 ..............
[CV]  n_estimators=2000, max_features=None, max_depth=20, score=0.926984126984127, total=   2.9s
[CV] n_estimators=2000, max_features=None, max_depth=20 ..............
[CV]  n_estimators=2000, max_features=None, max_depth=20, score=0.8945578231292517, total=   3.2s
[CV] n_estimators=2500, max_features=None, max_depth=20 ..............
[CV]  n_estimators=2500, max_features=None, max_depth=20, score=0.8539682539682539, total=   4.0s
[CV] n_estimators=2500, max_features=None, max_depth=20 ..

[CV]  n_estimators=2500, max_features=0.5, max_depth=None, score=0.891156462585034, total=   4.1s
[CV] n_estimators=3000, max_features=0.5, max_depth=None .............
[CV]  n_estimators=3000, max_features=0.5, max_depth=None, score=0.8444444444444444, total=   4.2s
[CV] n_estimators=3000, max_features=0.5, max_depth=None .............
[CV]  n_estimators=3000, max_features=0.5, max_depth=None, score=0.9777777777777777, total=   4.1s
[CV] n_estimators=3000, max_features=0.5, max_depth=None .............
[CV]  n_estimators=3000, max_features=0.5, max_depth=None, score=0.8253968253968254, total=   4.3s
[CV] n_estimators=3000, max_features=0.5, max_depth=None .............
[CV]  n_estimators=3000, max_features=0.5, max_depth=None, score=0.9365079365079364, total=   4.2s
[CV] n_estimators=3000, max_features=0.5, max_depth=None .............
[CV]  n_estimators=3000, max_features=0.5, max_depth=None, score=0.891156462585034, total=   3.8s
[CV] n_estimators=1000, max_features=0.8, max_depth=N

[CV]  n_estimators=1000, max_features=sqrt, max_depth=None, score=0.9174603174603175, total=   1.5s
[CV] n_estimators=1000, max_features=sqrt, max_depth=None ............
[CV]  n_estimators=1000, max_features=sqrt, max_depth=None, score=0.8945578231292517, total=   1.8s
[CV] n_estimators=2000, max_features=sqrt, max_depth=None ............
[CV]  n_estimators=2000, max_features=sqrt, max_depth=None, score=0.8507936507936508, total=   2.8s
[CV] n_estimators=2000, max_features=sqrt, max_depth=None ............
[CV]  n_estimators=2000, max_features=sqrt, max_depth=None, score=0.9714285714285714, total=   2.7s
[CV] n_estimators=2000, max_features=sqrt, max_depth=None ............
[CV]  n_estimators=2000, max_features=sqrt, max_depth=None, score=0.8174603174603176, total=   2.7s
[CV] n_estimators=2000, max_features=sqrt, max_depth=None ............
[CV]  n_estimators=2000, max_features=sqrt, max_depth=None, score=0.9047619047619048, total=   2.7s
[CV] n_estimators=2000, max_features=sqrt, ma

[Parallel(n_jobs=1)]: Done 560 out of 560 | elapsed: 35.0min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [1000, 2000, 2500, 3000], 'max_features': [0.5, 0.8, None, 'sqrt'], 'max_depth': [2, 3, 5, 10, 15, 20, None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=3)

In [340]:
grid_search2.fit(X_test, y_test)

Fitting 5 folds for each of 84 candidates, totalling 420 fits
[CV] loss=deviance, n_estimators=1000, learning_rate=0.1, max_depth=2 
[CV]  loss=deviance, n_estimators=1000, learning_rate=0.1, max_depth=2, score=0.8222222222222223, total=   1.4s
[CV] loss=deviance, n_estimators=1000, learning_rate=0.1, max_depth=2 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s remaining:    0.0s


[CV]  loss=deviance, n_estimators=1000, learning_rate=0.1, max_depth=2, score=0.8952380952380953, total=   0.3s
[CV] loss=deviance, n_estimators=1000, learning_rate=0.1, max_depth=2 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.8s remaining:    0.0s


[CV]  loss=deviance, n_estimators=1000, learning_rate=0.1, max_depth=2, score=0.8126984126984127, total=   0.3s
[CV] loss=deviance, n_estimators=1000, learning_rate=0.1, max_depth=2 
[CV]  loss=deviance, n_estimators=1000, learning_rate=0.1, max_depth=2, score=0.8888888888888888, total=   0.2s
[CV] loss=deviance, n_estimators=1000, learning_rate=0.1, max_depth=2 
[CV]  loss=deviance, n_estimators=1000, learning_rate=0.1, max_depth=2, score=0.8639455782312925, total=   0.2s
[CV] loss=deviance, n_estimators=2000, learning_rate=0.1, max_depth=2 
[CV]  loss=deviance, n_estimators=2000, learning_rate=0.1, max_depth=2, score=0.8222222222222223, total=   0.6s
[CV] loss=deviance, n_estimators=2000, learning_rate=0.1, max_depth=2 
[CV]  loss=deviance, n_estimators=2000, learning_rate=0.1, max_depth=2, score=0.8507936507936509, total=   0.7s
[CV] loss=deviance, n_estimators=2000, learning_rate=0.1, max_depth=2 
[CV]  loss=deviance, n_estimators=2000, learning_rate=0.1, max_depth=2, score=0.80634

...

[CV]  loss=deviance, n_estimators=2000, learning_rate=0.01, max_depth=5, score=0.9081632653061226, total=   1.5s
[CV] loss=deviance, n_estimators=1000, learning_rate=0.01, max_depth=10 
[CV]  loss=deviance, n_estimators=1000, learning_rate=0.01, max_depth=10, score=0.8571428571428572, total=   0.7s
[CV] loss=deviance, n_estimators=1000, learning_rate=0.01, max_depth=10 
[CV]  loss=deviance, n_estimators=1000, learning_rate=0.01, max_depth=10, score=0.9333333333333333, total=   0.8s
[CV] loss=deviance, n_estimators=1000, learning_rate=0.01, max_depth=10 
[CV]  loss=deviance, n_estimators=1000, learning_rate=0.01, max_depth=10, score=0.8476190476190476, total=   0.7s
[CV] loss=deviance, n_estimators=1000, learning_rate=0.01, max_depth=10 
[CV]  loss=deviance, n_estimators=1000, learning_rate=0.01, max_depth=10, score=0.9, total=   0.9s
[CV] loss=deviance, n_estimators=1000, learning_rate=0.01, max_depth=10 
[CV]  loss=deviance, n_estimators=1000, learning_rate=0.01, max_depth=10, score=0

[CV]  loss=exponential, n_estimators=1000, learning_rate=0.01, max_depth=2, score=0.9079365079365078, total=   0.3s
[CV] loss=exponential, n_estimators=1000, learning_rate=0.01, max_depth=2 
[CV]  loss=exponential, n_estimators=1000, learning_rate=0.01, max_depth=2, score=0.9013605442176871, total=   0.3s
[CV] loss=exponential, n_estimators=2000, learning_rate=0.01, max_depth=2 
[CV]  loss=exponential, n_estimators=2000, learning_rate=0.01, max_depth=2, score=0.8333333333333334, total=   0.6s
[CV] loss=exponential, n_estimators=2000, learning_rate=0.01, max_depth=2 
[CV]  loss=exponential, n_estimators=2000, learning_rate=0.01, max_depth=2, score=0.9111111111111112, total=   0.5s
[CV] loss=exponential, n_estimators=2000, learning_rate=0.01, max_depth=2 
[CV]  loss=exponential, n_estimators=2000, learning_rate=0.01, max_depth=2, score=0.8349206349206351, total=   0.5s
[CV] loss=exponential, n_estimators=2000, learning_rate=0.01, max_depth=2 
[CV]  loss=exponential, n_estimators=2000, le

[CV]  loss=exponential, n_estimators=2000, learning_rate=0.01, max_depth=15, score=0.9333333333333333, total=   1.1s
[CV] loss=exponential, n_estimators=2000, learning_rate=0.01, max_depth=15 
[CV]  loss=exponential, n_estimators=2000, learning_rate=0.01, max_depth=15, score=0.8428571428571429, total=   1.0s
[CV] loss=exponential, n_estimators=2000, learning_rate=0.01, max_depth=15 
[CV]  loss=exponential, n_estimators=2000, learning_rate=0.01, max_depth=15, score=0.8777777777777779, total=   1.1s
[CV] loss=exponential, n_estimators=2000, learning_rate=0.01, max_depth=15 
[CV]  loss=exponential, n_estimators=2000, learning_rate=0.01, max_depth=15, score=0.9149659863945578, total=   1.0s
[CV] loss=exponential, n_estimators=1000, learning_rate=0.01, max_depth=20 
[CV]  loss=exponential, n_estimators=1000, learning_rate=0.01, max_depth=20, score=0.8746031746031746, total=   0.8s
[CV] loss=exponential, n_estimators=1000, learning_rate=0.01, max_depth=20 
[CV]  loss=exponential, n_estimator

[Parallel(n_jobs=1)]: Done 420 out of 420 | elapsed:  4.7min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'loss': ['deviance', 'exponential'], 'n_estimators': [1000, 2000], 'learning_rate': [0.1, 0.05, 0.01], 'max_depth': [2, 3, 5, 10, 15, 20, None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=3)

In [338]:
grid_search1.best_score_

0.9091513700452248

In [339]:
grid_search1.best_estimator_ 

  #n_estimators=1000, max_features=None, max_depth=3, score=0.971428571428571
     #n_estimators=1000, max_features=0.5, max_depth=5, score=0.9904761904761905

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features=0.8, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=3000, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [341]:
grid_search2.best_score_

0.8959297685554668

In [154]:
grid_search1.best_score_

0.8960361798350626

In [342]:
grid_search2.best_params_

 #loss=exponential, n_estimators=1000, learning_rate=0.01, max_depth=5

{'learning_rate': 0.05,
 'loss': 'deviance',
 'max_depth': 10,
 'n_estimators': 1000}

In [279]:
#Predicción

#X_test = test.drop('Survived', axis=1).values


In [227]:
#clf=RandomForestClassifier(n_jobs=-1, n_estimators=2000, max_features='sqrt', max_depth=3)

clf=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features=0.8, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=2500, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
clf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features=0.8, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=2500, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [343]:
clf2=GradientBoostingClassifier(learning_rate= 0.05, loss= 'deviance',max_depth= 10,n_estimators= 1000)
clf2.fit(X_train,y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.05, loss='deviance', max_depth=10,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=1000,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [347]:
clf3=RandomForestClassifier(n_estimators=1000, max_features=0.5, max_depth=5)
clf3.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features=0.5, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [280]:
ruta_test='C:/Users/Juan Diego Bernate V/Downloads/Python/Practica/Titanic/test.csv'
Test=pd.read_csv(ruta_test)
Test.head()
Test.isnull().sum()


PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [281]:
Test['titulo'] = Test.Name.apply(lambda name: name.split(',')[1].split('.')[0].strip())

In [282]:
Test.titulo.unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Ms', 'Col', 'Rev', 'Dr', 'Dona'],
      dtype=object)

In [283]:
#Corregimos los titulos 

titulos_arregladosT = {
    "Capt":       "Oficial",
    "Col":        "Oficial",
    "Major":      "Oficial",
    "Jonkheer":   "Realeza",
    "Don":        "Realeza",
    "Sir" :       "Realeza",
    "Dr":         "Oficial",
    "Rev":        "Oficial",
    "the Countess":"Realeza",
    "Dona":       "Realeza",
    "Mme":        "Senora",
    "Mlle":       "Senorita",
    "Ms":         "Senora",
    "Mr" :        "Senor",
    "Mrs" :       "Senora",
    "Miss" :      "Senorita",
    "Master" :    "Master",
    "Lady" :      "Realeza"
}

#Reemplazamos en el dataset unido
Test.titulo = Test.titulo.map(titulos_arregladosT)

In [284]:
Test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
titulo           0
dtype: int64

In [285]:
Test.Age = group.Age.apply(lambda x: x.fillna(x.median()))
Test['Fare']=Test['Fare'].fillna(Test['Fare'].median())
Test['Embarked'].fillna(Test['Embarked'].mode()[0], inplace=True)


In [286]:
dT = {1:'1st',2:'2nd',3:'3rd'} 
Test['Pclass'] = Test['Pclass'].map(dT)

In [287]:
CategoricasT = Test[['Pclass','Sex','Embarked','titulo']]
dummiesT = pd.get_dummies(CategoricasT,drop_first=True)
Test=Test.drop(['Pclass','Sex','Embarked','titulo'],1)

In [288]:
Test.isnull().sum()

PassengerId      0
Name             0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
dtype: int64

In [294]:
Test.head()

,PassengerId,Age,SibSp,Parch,Fare,familia,Pclass_2nd,Pclass_3rd,Sex_male,Embarked_Q,Embarked_S,titulo_Oficial,titulo_Realeza,titulo_Senor,titulo_Senora,titulo_Senorita
0,892,22.0,0,0,7.8292,1,0,1,1,1,0,0,0,1,0,0
1,893,38.0,1,0,7.0000,2,0,1,0,0,1,0,0,0,1,0
2,894,26.0,0,0,9.6875,1,1,0,1,1,0,0,0,1,0,0
3,895,35.0,0,0,8.6625,1,0,1,1,0,1,0,0,1,0,0
4,896,35.0,1,1,12.2875,3,0,1,0,0,1,0,0,0,1,0


In [289]:
Test['familia']=Test['SibSp']+Test['Parch']+1

In [291]:
Test=pd.concat([Test,dummiesT],axis=1)

In [297]:
Test=Test.drop(['Name','Cabin','Ticket'],1)


In [298]:
idpassenger=Test.PassengerId
Test=Test.drop(['PassengerId'],1)

In [301]:
Pred = clf.predict(Test)

In [344]:
Pred2 = clf2.predict(Test)

In [348]:
Pred3 = clf3.predict(Test)

In [304]:
prueba=np.array(idpassenger)

In [305]:
prueba

array([ 892,  893,  894,  895,  896,  897,  898,  899,  900,  901,  902,
        903,  904,  905,  906,  907,  908,  909,  910,  911,  912,  913,
        914,  915,  916,  917,  918,  919,  920,  921,  922,  923,  924,
        925,  926,  927,  928,  929,  930,  931,  932,  933,  934,  935,
        936,  937,  938,  939,  940,  941,  942,  943,  944,  945,  946,
        947,  948,  949,  950,  951,  952,  953,  954,  955,  956,  957,
        958,  959,  960,  961,  962,  963,  964,  965,  966,  967,  968,
        969,  970,  971,  972,  973,  974,  975,  976,  977,  978,  979,
        980,  981,  982,  983,  984,  985,  986,  987,  988,  989,  990,
        991,  992,  993,  994,  995,  996,  997,  998,  999, 1000, 1001,
       1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012,
       1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023,
       1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034,
       1035, 1036, 1037, 1038, 1039, 1040, 1041, 10

In [315]:
resultados=pd.DataFrame({'PassengerId':prueba, 'Survived':Pred})
resultados.to_csv('titanic_resultado2.csv',index=False)

In [345]:
resultados2=pd.DataFrame({'PassengerId':prueba, 'Survived':Pred2})
resultados2.to_csv('titanic_resultado3.csv',index=False)

In [349]:
resultados3=pd.DataFrame({'PassengerId':prueba, 'Survived':Pred3})
resultados3.to_csv('titanic_resultado4.csv',index=False)